# CS246 - Colab 3
## K-Means & PCA

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=095f0697eabf7cfb52c9e192fbfa022b8d25869806b92b5f53d8a243fb99a328
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we import some of the libraries usually needed by our workload.





In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Preprocessing

In this Colab, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [4]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

For convenience, given that the dataset is small, we first

*   construct a Pandas dataframe
*   tune the schema
*   and convert it into a Spark dataframe.

In [5]:
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [6]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

### Your task

If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2** and seed to **1**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering, here we use squared euclidean distance).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Set k = 2, seed = 1
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(df)

# Evaluate clustering by computing Silhouette score
predictions = model.transform(df)
silhouette = ClusteringEvaluator().evaluate(predictions)

In [8]:
silhouette

0.8342904262826145

Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other), please use the best case scenario since the output cluster ids can be a permutation of labels.

*HINT*: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two series.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [9]:
# Create series from spark dataframe
predictions_series = predictions.select(col("prediction")).toPandas()["prediction"]

In [10]:
# Count matches
np.count_nonzero(labels == predictions_series)

83

In [11]:
# Binary reverse on prediction column
reversed_predictions = predictions.withColumn('reversed_prediction', when(predictions['prediction'] == 1, 0).otherwise(1)).select(col('reversed_prediction'))
reversed_predictions_series = reversed_predictions.select(col("reversed_prediction")).toPandas()["reversed_prediction"]

In [12]:
np.count_nonzero(labels == reversed_predictions_series)

486

Now perform dimensionality reduction on the ```features``` using the [PCA](https://spark.apache.org/docs/latest/ml-features.html#pca) statistical procedure, available as well in MLlib.

Set the ```k``` parameter to **2**, effectively reducing the dataset size of a **15X** factor.

In [13]:
''' 6 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.
The running time should be less than 30 seconds. Sanity check: the fourth row in the result should be [-692.6905100570509,38.57692259208171]'''
# YOUR CODE HERE
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model_pca = pca.fit(features)

pcaFeatures = model_pca.transform(features).select("pcaFeatures").withColumnRenamed("pcaFeatures", "features")
pcaFeatures.show(truncate=False)

+-----------------------------------------+
|features                                 |
+-----------------------------------------+
|[-2260.013886292542,-187.96030122263687] |
|[-2368.9937557820544,121.58742425815537] |
|[-2095.6652015478608,145.11398565870124] |
|[-692.6905100570509,38.57692259208171]   |
|[-2030.2124927427067,295.2979839927931]  |
|[-888.2800535760762,26.079796157025662]  |
|[-1921.0822124748454,58.807572473099455] |
|[-1074.7813350047968,31.771227808469558] |
|[-908.5784781618834,63.83075279044635]   |
|[-861.5784494075684,40.57073549705316]   |
|[-1404.5591306499475,88.23218257736251]  |
|[-1524.2324408687823,-3.2630573167779313]|
|[-1734.385647746416,273.16267815114594]  |
|[-1162.914003223036,217.6348180834464]   |
|[-903.4301030498837,135.6151766608479]   |
|[-1155.8759954206853,76.8088938374218]   |
|[-1335.7294321308073,-2.4684005450354585]|
|[-1547.2640922523092,3.805675972574516]  |
|[-2714.964765181216,-164.37610864258835] |
|[-908.2502671870881,118.2164200

Now run K-means with the same parameters as above, but on the ```pcaFeatures``` produced by the PCA reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly (also the best case scenario).

In [14]:
''' 11-13 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
kmeans = KMeans().setK(2).setSeed(1)
model_kmeans = kmeans.fit(pcaFeatures)

# Evaluate clustering by computing Silhouette score
predictions_pca = model_kmeans.transform(pcaFeatures)
silhouette_pca = ClusteringEvaluator().evaluate(predictions_pca)

In [15]:
silhouette_pca

0.8348610363444832

In [16]:
# Create series from spark dataframe
predictions_series_pca = predictions_pca.select(col("prediction")).toPandas()["prediction"]

In [17]:
# Count matches
np.count_nonzero(labels == predictions_series_pca)

83

In [18]:
# Binary reverse on prediction column
reversed_predictions_pca = predictions_pca.withColumn('reversed_prediction', when(predictions_pca['prediction'] == 1, 0).otherwise(1)).select(col('reversed_prediction'))
reversed_predictions_series_pca = reversed_predictions_pca.select(col("reversed_prediction")).toPandas()["reversed_prediction"]

In [19]:
np.count_nonzero(labels == reversed_predictions_series_pca)

486

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!